# Agno Agents SDK Demo

[Agno](https://docs.agno.com/introduction) is a lightweight library for building Multimodal Agents.

Install the dependencies library:

In [1]:
%pip install -qU agno==1.2.6 python-dotenv ipywidgets

Note: you may need to restart the kernel to use updated packages.


## Setup LLM and Agent

Set up the LLM and Agent with the following code:

In [2]:
import os
from dotenv import load_dotenv
from getpass import getpass

# Load environment variables from .env file
load_dotenv()

# Get API base URL and model with default values
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE", "https://api.openai.com/v1")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")

# Set up OpenAI API configuration
# Try to get API key from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# If not found, ask user to input it
if not OPENAI_API_KEY:
    OPENAI_API_KEY = getpass("Enter your OpenAI API key:")

OPENAI_MODEL

'qwen2.5:7b'

Set up an agent:

In [3]:
from typing import Iterator
from agno.agent import Agent, RunResponse
from agno.models.openai import OpenAIChat
from agno.utils.pprint import pprint_run_response

model = OpenAIChat(
    id=OPENAI_MODEL,
    api_key=OPENAI_API_KEY,
    base_url=OPENAI_API_BASE
)

agent = Agent(
    model=model,
    description="You're a helpful assistant.",
    name="Assistant",
    markdown=True,
)

## Running an Agent

If you want to simply run an agent, you can use the agent's print_response method, pass in a prompt, and set stream=True to quickly see the returned results:

In [9]:
agent.print_response("Tell me a short story", stream=True)

Output()

You can use the `Agent.run()` method to run an agent and return a `RunResponse` object or a streaming `RunResponse` object.


In [10]:

response: RunResponse = agent.run("Tell me a short story")
pprint_run_response(response, markdown=True)

Output()

In [ ]:
response_stream: Iterator[RunResponse] = agent.run("Tell me a short story", stream=True)
pprint_run_response(response_stream, markdown=True)